In [ ]:
import yfinance as yf
import requests
import bs4 as bs
import datetime as dt
import pandas as pd
import io
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns

sns.set(style='darkgrid', context='talk', palette='Dark2')

from sklearn.preprocessing import OneHotEncoder


https://wire.insiderfinance.io/how-to-get-all-stocks-from-the-s-p500-in-python-fbe5f9cb2b61 used to get a list of s&p500 companies and their associated industry.

In [ ]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies') #web page we are looking at
soup = bs.BeautifulSoup(resp.text, 'lxml') # we are going to parse it as lxml
table = soup.find('table', {'class': 'wikitable sortable'}) #find the wiki table on the page

tickers = []
industry = []

for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    ind = row.findAll('td')[2].text
    tickers.append(ticker)
    industry.append(ind)
    tickers = [s.replace('\n', "") for s in tickers]

In [ ]:
ti = pd.DataFrame(tickers)
ti.columns = ['ticker']
ind = pd.DataFrame(industry)
ind.columns = ['industry']
dat = ti.join(ind)

Download daily price data for s&p500 companies using yfinance

In [ ]:
data = yf.download(tickers, start="2020-01-06", end="2023-06-30", interval = '1wk')
df = data.stack().reset_index().rename(index=str, columns={"level_1": "Symbol"}).sort_values(['Symbol','Date'])
df.reset_index()
df.head()

[*********************100%***********************]  503 of 503 completed

ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
ERROR:yfinance:['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1wk 2020-01-06 -> 2023-06-30)')


,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
0,2020-01-06,A,85.549835,87.589996,88.239998,83.599998,84.000000,8855200.0
496,2020-01-13,A,88.020920,90.120003,90.279999,86.699997,87.809998,10250000.0
992,2020-01-20,A,86.204269,88.260002,90.639999,87.580002,89.800003,6756000.0
1488,2020-01-27,A,80.637001,82.559998,88.360001,82.339996,86.540001,10528800.0
1984,2020-02-03,A,81.232796,83.169998,85.500000,82.110001,83.290001,8224000.0


In [ ]:
full = df.merge(dat,how='left', left_on = 'Symbol', right_on = 'ticker')
full.head()

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,ticker,industry
0,2020-01-06,A,85.549835,87.589996,88.239998,83.599998,84.000000,8855200.0,A,Health Care
1,2020-01-13,A,88.020920,90.120003,90.279999,86.699997,87.809998,10250000.0,A,Health Care
2,2020-01-20,A,86.204269,88.260002,90.639999,87.580002,89.800003,6756000.0,A,Health Care
3,2020-01-27,A,80.637001,82.559998,88.360001,82.339996,86.540001,10528800.0,A,Health Care
4,2020-02-03,A,81.232796,83.169998,85.500000,82.110001,83.290001,8224000.0,A,Health Care


In [ ]:
full.to_csv('yfinance.csv')